# 4.4. Operaciones de combinar, juntar y agrupar.

In [ ]:
import pandas as pd
import numpy as np

## Fusión de estructuras

Hay dos formas de fusionar estructuras de datos: 
 * Realizando cruces entre las estructuras (mediante las claves coincidentes de sus índices) 
 * Concatenando sus contenidos (bien por filas o columnas).

#### Función merge - JOIN de estructuras

Creamos dos DF para unirlos

In [ ]:
peliculas = pd.DataFrame(
            {'Año':[2014, 2014, 2013, 2013], 
             'Valoración':[6, None, 8.75, None],
             'Presupuesto':[160, 250, 100, None],
             'Director':['Peter Jackson', 'Gareth Edwards', 'Martin Scorsese', 'Alfonso Cuarón'],
             'Título':['Godzilla', 'El Hobbit III', 'El lobo de Wall Street', 'Gravity']}
)
peliculas

In [ ]:
directores = pd.DataFrame(
            {'Director':['Gareth Edwards', 'Martin Scorsese', 'Pedro Almodovar'],
             'AñoNacimiento':[1975, 1942, 1949],
             'Nacionalidad': ['England', 'USA', 'Spain']
             }
)
directores

Realizamos un merge de ambos DF (en este caso, las columnas que coinciden son las columnas de director)

In [ ]:
pd.merge(peliculas, directores)

La función merge busca, por defecto, aquellas claves de columnas que coinciden y realiza el cruce, eliminando del resultado aquellas filas para las que el cruce no es posible.

También podemos especificar, explícitamente, el conjunto de columnas a utilizar en el cruce.

In [ ]:
directores.columns = ['Nombre', 'Nacimiento', 'Nacionalidad'] # Cambio el nombre para que no coincidan en ambos DF

pd.merge(peliculas, directores, left_on='Director', right_on='Nombre') # Al especificar el merge, indico las columnas por las que hacer la comparativa

Al igual que ocurre en os JOIN de SQL, podemos especificar el modo de cruce a aplicar.

Haciendo que las filas de la estructura de la izquierda, derecha o ambas que no coincidan se mantengan en el resultado, estableciendo valores NaN en aquellos elementos para los que no exista información.

Mantengo todos los datos del DF de películas

In [ ]:
pd.merge(peliculas, directores, left_on='Director', right_on='Nombre', how='left')

Mantengo todos los datos del DF directores

In [ ]:
pd.merge(peliculas, directores, left_on='Director', right_on='Nombre', how='right')

Mantengo todos los datos de ambos DF (outer)

In [ ]:
pd.merge(peliculas, directores, left_on='Director', right_on='Nombre', how='outer')

#### Función concat

Concat nos permite fusionar estructuras (concatena) sin realizar ningún tipo de cruce entre ellas, sino "colocándolas" juntas para la creación de una estructura mayor. 

Podemos hacerlo tanto en filas como en columnas.

Por defecto, se concatenan filas y se mantienen las columnas de ambas estructuras, aunque no coincidan en clave, dejando a NaN los elementos que no existan.

In [ ]:
peliculas = pd.DataFrame(
            {'Año':[2014, 2014, 2013, 2013], 
             'Valoración':[6, None, 8.75, None],
             'Presupuesto':[160, 250, 100, None],
             'Director':['Peter Jackson', 'Gareth Edwards', 'Martin Scorsese', 'Alfonso Cuarón']},
            index = ['Godzilla', 'El Hobbit III', 'El lobo de Wall Street', 'Gravity']
)
peliculas

In [ ]:
peliculas2 = pd.DataFrame(
            {'Año':[2014, 2014], 
             'Valoración':[7.3, 6.3],
             'Director':['Evan Goldberg', ' Rupert Wyatt']},
            index = ['La entrevista', 'El jugador']
)
peliculas

In [ ]:
pd.concat([peliculas, peliculas2])

Al contrario que en R, no es necesario que ambos DF tengan el mismo número de filas, o columnas, para hacer la concatenación. En python, la concatenación se hace por el nombre del índice de filas o columnas.

Concatenando por columnas:

In [ ]:
peliculas3 = pd.DataFrame(
            {'Recaudación':[525, 722, 392]},
            index = ['Godzilla', 'El Hobbit III', 'El lobo de Wall Street']
)
peliculas3

In [ ]:
pd.concat([peliculas, peliculas3],axis=1)

Pandas nos permite eliminar del resultado aquellas combinaciones para las que no existen datos en alguna de las dos estructuras (la fila Gravity no está en películas3)

In [ ]:
pd.concat([peliculas, peliculas3], axis=1, join='inner')

Por último, puede ser útil identificar en la estructura resultante el origen de cada una de las filas para posterior análisis. 

La función concat incluye un parámetro <b>keys</b> que podemos utilizar para añadir una clave a cada uno de las estructuras origen, que se convertirá en el nivel más agregado de un índice jerárquico.

In [ ]:
pd.concat([peliculas, peliculas2], keys=['dataset1','dataset2'])

No se suelen usar índices jerárquicos porque son muy lentos a la hora de realizar consultas

## Operaciones de agrupación

Una de las funcionalidades que más se utilizan es hacer la agrupación de resultados y operaciones

Al estilo de las sentencias GROUP BY de SQL. 

La librería pandas incluye esta funcionalidad.

In [ ]:
peliculas

In [ ]:
agrupado = peliculas.groupby('Año')
agrupado

Una agrupación es un iterador.

Es una representación interna del conjunto de registros que pertenecen a cada grupo.

Sirve para aplicar alguna operación sobre dichos grupos o para iterar sobre ellos.

<center>
<img src="imgs/group.png"  alt="drawing" width=400"/>
</center>
Las siguientes funciónes estan optimizadas para su aplicación:
<center>
<img src="imgs/group_method.png"  alt="drawing" width=500"/>
</center>                                                                                           

In [ ]:
peliculas

Calculamos la media para cada grupo

In [ ]:
agrupado.mean()

Habíamos agrupado por año. Para la columna de directores no se puede calcular la media. Hemos realizado el cálculo sobre el resto de columnas (valoración y presupuesto).

Podemos realizar el conteo de valores no nulos por grupo:

In [ ]:
agrupado.count()

En este caso sí que podemos realizar la operación también sobre la columna de director

Podemos realizar la agrupación por múltiples claves.

In [ ]:
peliculas.groupby(['Año', 'Director']).sum()

Podemos hacer que una función, definida por nosotros, establezca el criterio de agrupación.

In [ ]:
def titulo_largo(elemento):
    if len(elemento) > 10:
        return "Largo"
    else:
        return "Corto"
    
peliculas.groupby(titulo_largo).sum()

El elemento que recibe la función "titulo_largo" es el índice por filas del DF

Podemos usar una función específica con apply()

In [ ]:
def custom_fun(x):
    x['Director'] = x['Director'].str.upper() # El x que recibe es un DF con cada fila    
    return x

peliculas.groupby('Año').apply(custom_fun)

Podemos iterar sobre los grupos

In [ ]:
for year, group in peliculas.groupby('Año'):
    print(year)
    print()
    print(group)
    print()

Podemos generar un diccionario

In [ ]:
pieces = dict(list(peliculas.groupby('Año')))
pieces

Al cual podemos consultar por key

In [ ]:
pieces[2013]

## Tablas pivote
Pandas incluye la posibilidad de gestionar los DF, como una Pivot Table de Excel.

Los parámetros de la función son los siguientes:
<center>
<img src="imgs/pivot_methods.png"  alt="drawing" width=700"/>
</center>  

Generamos un DF para "interrogar"

In [ ]:
peliculas = pd.DataFrame({
    'Año':[2014, 2014, 2013, 2013], 
    'Valoración':[6, None, 8.75, None],
    'Presupuesto':[160, 250, 100, None],
    'Director':['Peter Jackson', 'Gareth Edwards', 'Martin Scorsese', 'Alfonso Cuarón'],
    'Título':['Godzilla', 'El Hobbit III', 'El lobo de Wall Street', 'Gravity']
})
peliculas

Le indicamos la información que estará en los ejes, y los valores que contendrá la tabla

In [ ]:
peliculas.pivot(index='Año', columns='Director', values='Título')

Podemos pivotar la tabla, exáctamente igual que en excel

In [ ]:
peliculas.pivot(index='Director', columns='Año', values='Presupuesto')

Otra manera de hacer lo mismo

In [ ]:
pd.pivot_table(peliculas, index='Director', columns='Año', values='Presupuesto')

También podemos crear tablas pivote utilizando una función de agregación para los valores, de forma que se haga una agrupación de resultados.

In [ ]:
peliculas['Valoración'] = [6, 6, 5, 5]
peliculas

Si hay más de un registro que cumpla año & valoración, que haga la suma de los presupuestos

In [ ]:
pd.pivot_table(peliculas, 
               values='Presupuesto', 
               index=['Año'], 
               columns=['Valoración'], 
               aggfunc=np.sum)

___
# Ejercicios

**4.4.1.** Carga el fichero  train.csv.

**4.4.2.** Calula el número de pasajeros por clase.

**4.4.3.** Calula la edad media de los supervivientes y los no supervivientes.

**4.4.4.** Calula el porcentage de supervientes por edad.

**4.4.5.** Calula el porcentage de supervientes por edad en intervalos de 10 y 5 años.

**4.4.6.** Crea un gráfico de barras de los datos anteriores.

**4.4.7.** Crea una tabla pivote con supervivientes como índice, la clase como columnas y la media de edad.